## Analyzing Tweets

We can gather a sample of Twitter data using the Twitter API (https://dev.twitter.com).  To do so, we'll need to create a Twitter application and get credentials for it.  You can do this manually at https://app.twitter.com.  Once you have an app, go to the "Key and Access Tokens" tab to find your credentials.

In [34]:
import random
import twitter
import emoji
import itertools
import pandas as pd
from itertools import chain
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
import tensorflow.keras.callbacks
import json

import os
import nb_utils
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Embedding, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate, Average

from gensim.models import Word2Vec

In [17]:
# Fill these in!

CONSUMER_KEY = 'xbMuxcJpRTiVGt2C2EYnA'
CONSUMER_SECRET = '2DbQTsvIptkPTdaUcos8DDvQH9fzO0hNjJpUT2uVzQ'
ACCESS_TOKEN = '7319442-EDm4CPxL7W4KkZcGWRMJNVHp88W5OH9vgblu898fg'
ACCESS_SECRET = '5ZxJSbqXhG7uhgXzTFWf9XhkfsxxinlPRXyDTzbA9w'

In [18]:
auth=twitter.OAuth(
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    token=ACCESS_TOKEN,
    token_secret=ACCESS_SECRET,
)

status_stream = twitter.TwitterStream(auth=auth).statuses

list(itertools.islice(
    (x['text'] for x in status_stream.sample() if x.get('text') is not None),
    0,
    5
))
# [x['text'] for x in itertools.islice(status_stream.sample(), 0, 5) if x.get('text')]

['In 2018, the ladies of #thecollective landed 8 tv feature stories for our incredible clients TOTALING $347,793 in e… https://t.co/FxpnHeIQuQ',
 '@sakuraimaki3 merry Christmas✨🎄✨',
 'دعاء السجود:سبوح قدوس رب الملائكة والروح https://t.co/HpibZCgbwi',
 'こころ 配信中!!\nhttps://t.co/Izcwf8aNJV',
 'Vou comer pra caralho hoje Pprt']

In [19]:
status_stream = twitter.TwitterStream(auth=auth).statuses

def english_has_emoji(tweet):
    if tweet.get('lang') != 'en':
        return False
    return any(ch for ch in tweet.get('text', '') if ch in emoji.UNICODE_EMOJI)

%time tweets = list(itertools.islice(filter(english_has_emoji, status_stream.sample()), 0, 100))

CPU times: user 676 ms, sys: 68.5 ms, total: 745 ms
Wall time: 53.8 s


In [20]:
stripped = []
for tweet in tweets:
    text = tweet['text']
    emojis = {ch for ch in text if ch in emoji.UNICODE_EMOJI}
    if len(emojis) == 1:
        emoiji = emojis.pop()
        text = ''.join(ch for ch in text if ch != emoiji)
        stripped.append((text, emoiji))
len(stripped)

63

## Using the CNN

Let's see what the CNN of the previous chapter does on the data

In [21]:
all_tweets = pd.read_csv('data/emojis.txt')
all_tweets['emoji'].value_counts()

😂    20061
😭     8954
❤     6802
😍     4815
🔥     3470
🤣     3427
🤔     2391
🙏     2225
😊     2005
✨     1971
💕     1969
👀     1860
🎉     1785
😩     1769
💜     1718
🙄     1650
💀     1630
👏     1470
😘     1346
🖤     1334
🙌     1324
🎄     1302
👍     1242
💯     1204
😔     1154
😉     1118
💖     1068
😎     1063
💙     1059
😁      996
     ...  
🐃        1
🥙        1
🕗        1
📴        1
🌖        1
🚺        1
🏋        1
🕐        1
🚋        1
⚱        1
🌜        1
🏗        1
☸        1
🥉        1
↕        1
🈵        1
🎳        1
💺        1
🚝        1
⛩        1
↙        1
📑        1
⛑        1
🚥        1
🎽        1
➰        1
🐫        1
⛹        1
🔢        1
📗        1
Name: emoji, Length: 982, dtype: int64

In [22]:
tweets = all_tweets.groupby('emoji').filter(lambda c:len(c) > 1000)
tweets['emoji'].value_counts()

😂    20061
😭     8954
❤     6802
😍     4815
🔥     3470
🤣     3427
🤔     2391
🙏     2225
😊     2005
✨     1971
💕     1969
👀     1860
🎉     1785
😩     1769
💜     1718
🙄     1650
💀     1630
👏     1470
😘     1346
🖤     1334
🙌     1324
🎄     1302
👍     1242
💯     1204
😔     1154
😉     1118
💖     1068
😎     1063
💙     1059
Name: emoji, dtype: int64

In [23]:
tweets

,text,emoji
0,RT @missypbb_: Missy casually serving good loo...,😍
1,Yeah I just finished the heirs ️,❤
2,RT @JustinPTurner: I hurt my foot and my dog s...,😂
3,you ain't shit,😂
4,RT @HoustonMay23: Christmas just got better ht...,😂
5,how cute https://t.co/C9rFaFU2b7,👀
10,RT @SuchAqueent: “Send me a pic” is where a wo...,😂
11,Anna - Trio 1⃣2⃣3⃣ https://t.co/OWEZwo34BL\nA...,😍
12,RT @6blogger: I'll never get bored of sharing ...,😂
13,RT @youbs19: Fuck winter wonderland come we go...,🤔


In [24]:
max(tweets['text'], key=lambda t:len(t))

'@KristenScholer NYSE-800 &amp;NASDAQ-200MARKETCRASH%\n(FED RATES DECISION&amp;TRADE WITH CHINA AMERICAN&amp;WORLD OPPOSES&amp;BITCO… https://t.co/37efEyLeSr'

In [25]:
chars = list(sorted(set(chain(*tweets['text']))))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
max_sequence_len = max(len(x) for x in tweets['text'])

emojis = list(sorted(set(tweets['emoji'])))
emoji_to_idx = {em: idx for idx, em in enumerate(emojis)}
emojis[:10]

train_tweets, test_tweets = train_test_split(tweets, test_size=0.1)


In [26]:
def data_generator(tweets, batch_size):
    while True:
        if batch_size is None:
            batch = tweets
            batch_size = batch.shape[0]
        else:
            batch = tweets.sample(batch_size)
        X = np.zeros((batch_size, max_sequence_len, len(chars)))
        y = np.zeros((batch_size,))
        for row_idx, (_, row) in enumerate(batch.iterrows()):
            y[row_idx] = emoji_to_idx[row['emoji']]
            for ch_idx, ch in enumerate(row['text']):
                X[row_idx, ch_idx, char_to_idx[ch]] = 1
        yield X, y

next(data_generator(tweets, 10))

(array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],


In [27]:
def create_char_cnn_model(num_chars, max_sequence_len, num_labels):
    char_input = Input(shape=(max_sequence_len, num_chars), name='char_cnn_input')
    
    conv_1x = Conv1D(128, 6, activation='relu', padding='valid')(char_input)
    max_pool_1x = MaxPooling1D(4)(conv_1x)
    conv_2x = Conv1D(256, 6, activation='relu', padding='valid')(max_pool_1x)
    max_pool_2x = MaxPooling1D(4)(conv_2x)

    flatten = Flatten()(max_pool_2x)
    dense = Dense(128, activation='relu')(flatten)
    preds = Dense(num_labels, activation='softmax', name='char_cnn_predictions')(dense)

    model = Model(char_input, preds)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])
    return model

char_cnn_model = create_char_cnn_model(len(char_to_idx), max_sequence_len, len(emojis))
char_cnn_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_cnn_input (InputLayer)  [(None, 155, 1457)]       0         
_________________________________________________________________
conv1d (Conv1D)              (None, 150, 128)          1119104   
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 37, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 32, 256)           196864    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 8, 256)            0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dens

In [35]:
early = keras.callbacks.EarlyStopping(monitor='loss',
                              min_delta=0.03,
                              patience=2,
                              verbose=0, mode='auto')



BATCH_SIZE = 512
char_cnn_model.fit_generator(
    data_generator(train_tweets, batch_size=BATCH_SIZE),
    epochs=20,
    steps_per_epoch=len(train_tweets) // BATCH_SIZE,
    verbose=2,
    callbacks=[early]
)

Epoch 1/20
 - 179s - loss: 2.7882 - acc: 0.2700
Epoch 2/20
 - 170s - loss: 2.3591 - acc: 0.3804
Epoch 3/20
 - 164s - loss: 2.1329 - acc: 0.4356
Epoch 4/20
 - 165s - loss: 1.9182 - acc: 0.4891
Epoch 5/20
 - 165s - loss: 1.7825 - acc: 0.5231
Epoch 6/20
 - 169s - loss: 1.6476 - acc: 0.5584
Epoch 7/20
 - 164s - loss: 1.5344 - acc: 0.5873
Epoch 8/20
 - 164s - loss: 1.4285 - acc: 0.6120
Epoch 9/20
 - 166s - loss: 1.3437 - acc: 0.6336
Epoch 10/20
 - 169s - loss: 1.2503 - acc: 0.6594
Epoch 11/20
 - 169s - loss: 1.1598 - acc: 0.6839
Epoch 12/20
 - 166s - loss: 1.0705 - acc: 0.7065
Epoch 13/20
 - 165s - loss: 0.9823 - acc: 0.7314
Epoch 14/20
 - 168s - loss: 0.8942 - acc: 0.7544
Epoch 15/20
 - 161s - loss: 0.8119 - acc: 0.7774
Epoch 16/20
 - 169s - loss: 0.7379 - acc: 0.7981
Epoch 17/20
 - 170s - loss: 0.6767 - acc: 0.8144
Epoch 18/20
 - 166s - loss: 0.6115 - acc: 0.8328
Epoch 19/20
 - 164s - loss: 0.5498 - acc: 0.8487
Epoch 20/20
 - 165s - loss: 0.4878 - acc: 0.8655


In [37]:
char_cnn_model.evaluate_generator(
    data_generator(test_tweets, batch_size=BATCH_SIZE),
    steps=len(test_tweets) // BATCH_SIZE
)

[2.6949294060468674, 0.5369873]

In [39]:
os.makedirs('zoo/07', exist_ok=True)

with open('zoo/07/emoji_chars.json', 'w') as fout:
    json.dump({
        'emojis': ''.join(emojis),
        'char_to_idx': char_to_idx,
        'max_sequence_len': max_sequence_len,
    }, fout)
char_cnn_model.save('zoo/07/char_cnn_model.h5')
char_cnn_model.save_weights('zoo/07/char_cnn_model_weights.h5')

In [40]:
pd.options.display.max_colwidth = 128
inspect_tweets = test_tweets.sample(100)
predicted = char_cnn_model.predict_generator(data_generator(inspect_tweets, batch_size=None), steps=1)
show = pd.DataFrame({
    'text': inspect_tweets['text'],
    'true': inspect_tweets['emoji'],
    'pred': [emojis[np.argmax(x)] for x in predicted],
})
show = show[['text', 'true', 'pred']]
show.head(10)

,text,true,pred
44191,i’m really gonna try to be celibate for all of 2019.,😭,🙏
58270,@_NEObts CUTIE,💕,😂
67486,RT @Essence: 2018 was definitely the year of Black women in politics! 🏾 https://t.co/q3v8Ccxixt,🙌,🙏
17855,RT @powerfeen: I wish wealth upon me and all my friends so this can be our only problem https://t.co/WUi2Fx8aKR,😭,😭
48687,@realdawnsummers @AnthonyCurtis68 i thought that was still legal in Mississippi \n\nkidding!!!!! lol... that's Alabama,🤔,👍
60415,"RT @itsaustonpugh: when i put """" this is what i really mean https://t.co/zPuraXzTLE",👀,👀
97450,RT @fine_ass_mari: I hate texting.. come live with me,😂,😂
13183,@HesterDressage @Hezzzza Wow what a lucky girl ️,❤,❤
131561,I want to starve myself,😂,😂
62357,RT @mytweetsaremid: Tupac really killed it here https://t.co/Edt8dnDR7r,🔥,😭


In [41]:
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate

def create_char_cnn_model2(num_chars, max_sequence_len, num_labels, drop_out=0.25):
    char_input = Input(shape=(max_sequence_len, num_chars), name='char_cnn_input')
    
    layers = []
    for window in (4, 5, 6):
        conv_1x = Conv1D(128, window, activation='relu', padding='valid')(char_input)
        max_pool_1x = MaxPooling1D(4)(conv_1x)
        dropout_1x = Dropout(drop_out)(max_pool_1x)
        conv_2x = Conv1D(256, window, activation='relu', padding='valid')(dropout_1x)
        max_pool_2x = MaxPooling1D(4)(conv_2x)
        dropout_2x = Dropout(drop_out)(max_pool_2x)
        layers.append(dropout_2x)

    merged = Concatenate(axis=1)(layers)

    dropout = Dropout(drop_out)(merged)
    
    flatten = Flatten()(dropout)
    dense = Dense(128, activation='relu')(flatten)
    preds = Dense(num_labels, activation='softmax', name='char_cnn_predictions')(dense)

    model = Model(char_input, preds)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])
    return model

char_cnn_model2 = create_char_cnn_model2(len(char_to_idx), max_sequence_len, len(emojis))
char_cnn_model2.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_cnn_input (InputLayer)     [(None, 155, 1457)]  0                                            
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 152, 128)     746112      <tensorflow.python.keras.engine.i
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 151, 128)     932608      <tensorflow.python.keras.engine.i
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 150, 128)     1119104     <tensorflow.python.keras.eng

In [44]:
BATCH_SIZE = 2048
char_cnn_model2.fit_generator(
    data_generator(train_tweets, batch_size=BATCH_SIZE),
    epochs=30,
    steps_per_epoch=len(train_tweets) // BATCH_SIZE,
    verbose=2,
    callbacks=[early]
)

Epoch 1/30
 - 319s - loss: 2.9711 - acc: 0.2252
Epoch 2/30
 - 183s - loss: 2.8548 - acc: 0.2462
Epoch 3/30
 - 185s - loss: 2.7502 - acc: 0.2732
Epoch 4/30
 - 187s - loss: 2.5882 - acc: 0.3227
Epoch 5/30
 - 187s - loss: 2.4524 - acc: 0.3555
Epoch 6/30
 - 185s - loss: 2.3754 - acc: 0.3711
Epoch 7/30
 - 186s - loss: 2.3008 - acc: 0.3874
Epoch 8/30
 - 184s - loss: 2.2366 - acc: 0.4045
Epoch 9/30
 - 189s - loss: 2.1685 - acc: 0.4236
Epoch 10/30
 - 184s - loss: 2.0901 - acc: 0.4389
Epoch 11/30
 - 187s - loss: 2.0251 - acc: 0.4585
Epoch 12/30
 - 185s - loss: 1.9714 - acc: 0.4709
Epoch 13/30
 - 185s - loss: 1.9300 - acc: 0.4819
Epoch 14/30
 - 186s - loss: 1.8593 - acc: 0.4990
Epoch 15/30
 - 186s - loss: 1.8430 - acc: 0.5057
Epoch 16/30
 - 186s - loss: 1.7937 - acc: 0.5158
Epoch 17/30
 - 187s - loss: 1.7518 - acc: 0.5251
Epoch 18/30
 - 186s - loss: 1.7158 - acc: 0.5335
Epoch 19/30
 - 185s - loss: 1.6895 - acc: 0.5404
Epoch 20/30
 - 187s - loss: 1.6376 - acc: 0.5544
Epoch 21/30
 - 186s - loss: 1

In [46]:
char_cnn_model2.evaluate_generator(
    data_generator(test_tweets, batch_size=BATCH_SIZE),
    steps=len(test_tweets) // BATCH_SIZE
)

[1.7012174129486084, 0.55358887]

In [47]:
"75s - loss: 2.3855 - acc: 0.4368\n[2.8089022636413574, 0.38840296648550726]"

'75s - loss: 2.3855 - acc: 0.4368\n[2.8089022636413574, 0.38840296648550726]'

## Featurizing and preparing our data

Just like we did when computing word embeddings, we want to featurize our data so we can classify it effectively.

In [48]:
VOCAB_SIZE = 50000
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(tweets['text'])

In [49]:
training_tokens = tokenizer.texts_to_sequences(train_tweets['text'])
test_tokens = tokenizer.texts_to_sequences(test_tweets['text'])
max_num_tokens = max(len(x) for x in chain(training_tokens, test_tokens))
training_tokens = pad_sequences(training_tokens, maxlen=max_num_tokens)
test_tokens = pad_sequences(test_tokens, maxlen=max_num_tokens)

In [50]:
training_labels = np.asarray([emoji_to_idx[em] for em in train_tweets['emoji']])
test_labels = np.asarray([emoji_to_idx[em] for em in test_tweets['emoji']])

In [70]:
w2v = Word2Vec(train_tweets['text'], size=100, window=5, min_count=1, workers=4)
w2v.save('data/twitter_w2v.model')

In [82]:
def load_weights(tokenizer):
    w2v_model = Word2Vec.load('data/twitter_w2v.model')
    w2v = np.zeros((tokenizer.num_words, w2v_model.wv.syn0.shape[1]))
    for k, v in tokenizer.word_index.items():
        if v >= tokenizer.num_words:
            continue
        if k in w2v_model.wv:
            w2v[v] = w2v_model.wv[k]
    return w2v

In [83]:
# This may take a while to load

w2v = load_weights(tokenizer)

model = Word2Vec.load('data/twitter_w2v.model')
w2v = np.zeros((tokenizer.num_words, model.wv.syn0.shape[1]))
found = 0
for k, v in tokenizer.word_index.items():
    if v >= tokenizer.num_words:
        continue
    if k in model.wv:
        w2v[v] = model.wv[k]
        found += 1
found, tokenizer.num_words


(134, 50000)

# World Level CNN

As with our previous task, we can try using more powerful models to classify our text.  In this case, the limited training data and text size limit their effectiveness.

In [84]:
def create_cnn_model(vocab_size, embedding_size=None, embedding_weights=None, drop_out=0.2):
    message = Input(shape=(max_num_tokens,), dtype='int32', name='cnn_input')
    
    
    # The convolution layer in keras does not support masking, so we just allow
    # the embedding layer to learn an explicit value.
    embedding = Embedding(mask_zero=False, input_dim=vocab_size, 
                          output_dim=embedding_weights.shape[1], 
                          weights=[embedding_weights],
                          trainable=True,
                          name='cnn_embedding')(message)
    
    global_pools = []
    for window in 2, 3:
        conv_1x = Conv1D(128, window, activation='relu', padding='valid')(embedding)
        max_pool_1x = MaxPooling1D(2)(conv_1x)
        conv_2x = Conv1D(256, window, activation='relu', padding='valid')(max_pool_1x)
        max_pool_2x = MaxPooling1D(2)(conv_2x)
        conv_3x = Conv1D(256, window, activation='relu', padding='valid')(max_pool_2x)

        global_pools.append(GlobalMaxPooling1D()(conv_3x))

    merged = Concatenate(axis=1)(global_pools)
    fc1 = Dense(units=128, activation='elu')(merged)
    preds = Dense(units=len(emojis), activation='softmax', name='cnn_predictions')(fc1)
    model = Model(
        inputs=[message],
        outputs=[preds],
    )
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

cnn_model = create_cnn_model(VOCAB_SIZE, embedding_weights=w2v)
cnn_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cnn_input (InputLayer)          [(None, 44)]         0                                            
__________________________________________________________________________________________________
cnn_embedding (Embedding)       (None, 44, 100)      5000000     <tensorflow.python.keras.engine.i
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 43, 128)      25728       <tensorflow.python.keras.layers.e
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 42, 128)      38528       <tensorflow.python.keras.layers.e
__________________________________________________________________________________________________
max_poolin

In [85]:
cnn_model.fit(training_tokens, training_labels, epochs=5)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/5


InvalidArgumentError: Could not colocate node with its resource and reference inputs; devices /job:localhost/replica:0/task:0/device:CPU:0 and /job:localhost/replica:0/task:0/device:GPU:0 are not compatible.
	 [[{{node training_2/RMSprop/RMSprop/update/ResourceSparseApplyRMSProp}}]] [Op:StatefulPartitionedCall]

In [ ]:
def create_lstm_model(vocab_size, embedding_size=None, embedding_weights=None):
    message = Input(shape=(None,), dtype='int32', name='lstm_input')
    embedding = Embedding(mask_zero=False, input_dim=vocab_size, 
                          output_dim=embedding_weights.shape[1], 
                          weights=[embedding_weights],
                          trainable=True,
                          name='lstm_embedding')(message)

    lstm_1 = LSTM(units=128, return_sequences=False)(embedding)
    preds = Dense(units=len(emojis), activation='softmax', name='lstm_predictions')(lstm_1)
    
    model = Model(
        inputs=[message],
        outputs=[preds],
    )
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [ ]:
lstm_model = create_lstm_model(VOCAB_SIZE, embedding_weights=w2v)
lstm_model.summary()

In [ ]:
lstm_model.fit(training_tokens, training_labels, epochs=12, batch_size=1024, callbacks=[early])

In [ ]:
lstm_model.evaluate(test_tokens, test_labels)

## Comparing our models

Let's compare the predictions from our models on a sample of our data.

In [ ]:
test_char_vectors, _ = next(data_generator(test_tweets, None)) 

In [ ]:
predictions = {
    label: [emojis[np.argmax(x)] for x in pred]
    for label, pred in (
        ('lstm', lstm_model.predict(test_tokens[:100])),
        ('char_cnn', char_cnn_model.predict(test_char_vectors[:100])),
        ('cnn', cnn_model.predict(test_tokens[:100])),
    )
}

In [ ]:
# Make a dataframe just for test data
pd.options.display.max_colwidth = 128
test_df = test_tweets[:100].reset_index()
eval_df = pd.DataFrame({
    'content': test_df['text'],
    'true': test_df['emoji'],
    **predictions
})
eval_df[['content', 'true', 'char_cnn', 'cnn', 'lstm']].head(25)

## Qualitative Evaluation

We can examine some of our error cases by hand.  Often, the models tend to agree when they make mistakes, and that the mistakes aren't unreasonable: this task would be challenging even for a human.

In [ ]:
eval_df[eval_df['lstm'] != eval_df['true']].head(10)

In [ ]:
def combined_data_generator(tweets, tokens, batch_size):
    tweets = tweets.reset_index()
    while True:
        batch_idx = random.sample(range(len(tweets)), batch_size)
        tweet_batch = tweets.iloc[batch_idx]
        token_batch = tokens[batch_idx]
        char_vec = np.zeros((batch_size, max_sequence_len, len(chars)))
        token_vec = np.zeros((batch_size, max_num_tokens))
        y = np.zeros((batch_size,))
        for row_idx, (token_row, (_, tweet_row)) in enumerate(zip(token_batch, tweet_batch.iterrows())):
            y[row_idx] = emoji_to_idx[tweet_row['emoji']]
            for ch_idx, ch in enumerate(tweet_row['text']):
                char_vec[row_idx, ch_idx, char_to_idx[ch]] = 1
            token_vec[row_idx, :] = token_row
        yield {'char_cnn_input': char_vec, 'cnn_input': token_vec, 'lstm_input': token_vec}, y

d, y = next(combined_data_generator(train_tweets, training_tokens, 5))
d['lstm_input'].shape

In [ ]:
def prediction_layer(model):
    layers = [layer for layer in model.layers if layer.name.endswith('_predictions')]
    return layers[0].output

def create_ensemble(*models):
    inputs = [model.input for model in models]
    predictions = [prediction_layer(model) for model in models]
    merged = Average()(predictions)
    model = Model(
        inputs=inputs,
        outputs=[merged],
    )
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model


ensemble = create_ensemble(char_cnn_model2, cnn_model, lstm_model)
ensemble.summary()

In [ ]:
BATCH_SIZE = 512
ensemble.fit_generator(
    combined_data_generator(train_tweets, training_tokens, BATCH_SIZE),
    epochs=20,
    steps_per_epoch=len(train_tweets) / BATCH_SIZE,
    verbose=2,
    callbacks=[early]
)

In [ ]:
ensemble.evaluate_generator(
    combined_data_generator(test_tweets, test_tokens, BATCH_SIZE),
    steps=len(test_tweets) / BATCH_SIZE
)

In [ ]:
len(train_tweets)